In [ ]:
import zipfile
import os
import time

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file = '/content/drive/MyDrive/cbert_aug-crayon.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

['.config', '__pycache__', 'aug_data', 'cbert_utils.py', 'cbert_pretrained', 'drive', 'datasets', 'train_text_classifier.py', '.ipynb_checkpoints', 'utils.py', 'global.config', 'nets.py', 'args_of_text_classifier.py', 'cbert_augdata.py', 'text_classification', 'cbert_finetune.py', 'evaluator.py', 'triggers.py', 'README.md', 'sample_data']


In [ ]:
import torch

In [ ]:
!pip install transformers

In [ ]:
! pip install ipdb

In [ ]:
os.chdir('./')

In [ ]:
!ls

args_of_text_classifier.py  datasets	   README.md
aug_data		    drive	   sample_data
cbert_augdata.py	    evaluator.py   text_classification
cbert_finetune.py	    global.config  train_text_classifier.py
cbert_pretrained	    nets.py	   triggers.py
cbert_utils.py		    __pycache__    utils.py


In [ ]:
import pandas as pd


In [ ]:
sample_data = pd.read_csv('data (3).tsv', sep='\t')

FileNotFoundError: ignored

In [ ]:
sample_data.head(20)

,words,rst,class,segment,label,code_class
0,school uniforms is good. Or drag a student ove...,none,Argument_against,0,2,0
1,school uniforms is good.,attribution,Argument_against,1,0,0
2,school uniforms is good.,none,Argument_against,1,2,0
3,Only,attribution,Argument_against,1,0,0
4,for prostitutes is the school uniform an oblig...,none,Argument_against,1,2,0
5,why ) .,elaboration,Argument_against,1,1,0
6,school uniforms is good. The story currently r...,none,NoArgument,2,2,2
7,resulting in a large number of regular inquiri...,elaboration,NoArgument,2,1,2
8,"one on Google , Yahoo and Bing for the search ...",elaboration,NoArgument,2,1,2
9,"school uniforms is good. Some might say , Wait...",none,NoArgument,3,2,2


In [ ]:
import os
import csv
import logging
import random

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

"""initialize logger"""
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

class InputExample(object):
    """A single training/test example for simple sequence classification."""
    def __init__(self, guid, text_a, segment, cl, label=None):
        """Constructs a InputExample/

        Args:
            guid: Unique id for the example.
            text: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
        """
        self.guid = guid
        self.text_a = text_a
        self.label = label
        self.segment = segment
        self.cl = cl

class InputFeature(object):
    """A single set of features of data."""

    def __init__(self, init_ids, input_ids, input_mask, segment_ids, masked_lm_labels, segment_span, class_span):
        self.init_ids = init_ids
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.masked_lm_labels = masked_lm_labels
        self.segment_span = segment_span
        self.class_span = class_span

class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of 'InputExample's for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of 'InputExample's for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines

class AugProcessor(DataProcessor):
    """Processor for dataset to be augmented."""

    def get_train_examples(self, data_dir):
        """See base calss."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")
    
    def get_labels(self, name):
        """add your dataset here"""
        if name in ['stsa.binary', 'mpqa', 'rt-polarity', 'subj']:
            return ["0", "1"]
        elif name in ['stsa.fine']:
            return ["0", "1", "2", "3", "4"]
        elif name in ['TREC']:
            return ["0", "1", "2", "3", "4", "5"]
        elif name in ['amazon']:
            print("Amazon")
            return ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
  
    def _create_examples(self, lines, set_type):
        """Create examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            segment = line[-3]
            #print(segment)
            label = line[-2]
            #print(label)
            cl = line[-1]
            #print(cl)
            examples.append(
                InputExample(guid=guid, text_a=text_a, segment = segment, cl = cl, label=label))
        return examples

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of 'InputBatch's."""
    
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i
    
    features = []
    for (ex_index, example) in enumerate(examples):
        # The convention in BERT is:
        # tokens:   [CLS] is this jack ##son ##ville ? [SEP]
        # type_ids: 0     0  0    0    0     0       0 0    
        tokens_a = tokenizer._tokenize(example.text_a)
        tokens_label = label_map[example.label]
        tokens, init_ids, input_ids, input_mask, segment_ids, masked_lm_labels, segment_span, class_span = extract_features(tokens_a, tokens_label, example.segment, example.cl, max_seq_length, tokenizer)

        #print("where is a mistake, I wonder...")
        
        """convert label to label_id"""
        label_id = label_map[example.label]

        """consturct features"""
        features.append(
            InputFeature(
                init_ids=init_ids,        
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                masked_lm_labels=masked_lm_labels,
                segment_span = segment_span,
                class_span = class_span))

        """print examples"""
        if ex_index < 5:
            logger.info("[cbert] *** Example ***")
            logger.info("[cbert] guid: %s" % (example.guid))
            logger.info("[cbert] tokens: %s" % " ".join(
                [str(x) for x in tokens]))
            logger.info("[cbert] init_ids: %s" % " ".join([str(x) for x in init_ids]))
            logger.info("[cbert] input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("[cbert] input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info("[cbert] segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("[cbert] masked_lm_labels: %s" % " ".join([str(x) for x in masked_lm_labels]))
    return features

def construct_train_dataloader(train_examples, label_list, max_seq_length, train_batch_size, num_train_epochs, tokenizer, device):
    """construct dataloader for training data"""
    
    num_train_steps = None
    global_step = 0
    train_features = convert_examples_to_features(
        train_examples, label_list, max_seq_length, tokenizer)
    num_train_steps = int(len(train_features) / train_batch_size * num_train_epochs)

    #print(train_features)
    #for f in train_features:
    #  print(f.init_ids)
    
    all_init_ids = torch.tensor([f.init_ids for f in train_features], dtype=torch.long, device=device)
    #print([f.init_ids for f in train_features])
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long, device=device)
    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long, device=device)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long, device=device)
    all_masked_lm_labels = torch.tensor([f.masked_lm_labels for f in train_features], dtype=torch.long, device=device)
    segment_span = torch.tensor([int(f.segment_span) for f in train_features], dtype=torch.long, device=device)
    class_span = torch.tensor([int(f.class_span) for f in train_features], dtype=torch.long, device=device)
    
    tensor_dataset = TensorDataset(all_init_ids, all_input_ids, all_input_mask, 
        all_segment_ids, all_masked_lm_labels, segment_span, class_span)
    train_sampler = RandomSampler(tensor_dataset)
    train_dataloader = DataLoader(tensor_dataset, sampler=train_sampler, batch_size=train_batch_size)
    return train_features, num_train_steps, train_dataloader

def rev_wordpiece(str):
    """wordpiece function used in cbert"""
    
    #print(str)
    if len(str) > 1:
        for i in range(len(str)-1, 0, -1):
            if str[i] == '[PAD]':
                str.remove(str[i])
            elif len(str[i]) > 1 and str[i][0]=='#' and str[i][1]=='#':
                str[i-1] += str[i][2:]
                str.remove(str[i])
    return " ".join(str[1:-1])


def extract_features(tokens_a, tokens_label, segment, cl, max_seq_length, tokenizer):
    """extract features from tokens"""

    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0: (max_seq_length - 2)]
    
    tokens = []
    segment_ids = []
    tokens.append('[CLS]')
    segment_ids.append(tokens_label)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(tokens_label)
    tokens.append('[SEP]')
    segment_ids.append(tokens_label)

    ## construct init_ids for each example
    init_ids = convert_tokens_to_ids(tokens, tokenizer)

    ## construct input_ids for each example, we replace the word_id using 
    ## the ids of masked words (mask words based on original sentence)
    masked_lm_probs = 0.15
    max_predictions_per_seq = 20
    rng = random.Random(12345)
    original_masked_lm_labels = [-100] * max_seq_length
    (output_tokens, masked_lm_positions, 
    masked_lm_labels) = create_masked_lm_predictions(
            tokens, masked_lm_probs, original_masked_lm_labels, max_predictions_per_seq, rng, tokenizer)
    input_ids = convert_tokens_to_ids(output_tokens, tokenizer)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)
    
    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        init_ids.append(0)
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
    
    assert len(init_ids) == max_seq_length
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return tokens, init_ids, input_ids, input_mask, segment_ids, masked_lm_labels, segment, cl

def convert_tokens_to_ids(tokens, tokenizer):
    """Converts tokens into ids using the vocab."""
    ids = []
    for token in tokens:
        token_id = tokenizer._convert_token_to_id(token)
        ids.append(token_id)
    return ids

def create_masked_lm_predictions(tokens, masked_lm_probs, masked_lm_labels, 
                                 max_predictions_per_seq, rng, tokenizer):
    """Creates the predictions for the masked LM objective."""

    #vocab_words = list(tokenizer.vocab.keys())
    
    cand_indexes = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        cand_indexes.append(i)
    
    rng.shuffle(cand_indexes)
    len_cand = len(cand_indexes)
    output_tokens = list(tokens)
    num_to_predict = min(max_predictions_per_seq, 
                         max(1, int(round(len(tokens) * masked_lm_probs))))
    
    masked_lm_positions = []
    covered_indexes = set()
    for index in cand_indexes:
        if len(masked_lm_positions) >= num_to_predict:
            break
        if index in covered_indexes:
            continue
        covered_indexes.add(index)

        masked_token = None
        ## 80% of the time, replace with [MASK]
        if rng.random() < 0.8:
            masked_token = "[MASK]"
        else:
            ## 10% of the time, keep original
            if rng.random() < 0.5:
                masked_token = tokens[index]
            ## 10% of the time, replace with random word
            else:
                masked_token = tokens[cand_indexes[rng.randint(0, len_cand - 1)]]
                
        masked_lm_labels[index] = convert_tokens_to_ids([tokens[index]], tokenizer)[0]
        output_tokens[index] = masked_token
        masked_lm_positions.append(index)
    return output_tokens, masked_lm_positions, masked_lm_labels

In [ ]:
class Arguments(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, data_dir="datasets", output_dir="aug_data", save_model_dir = "cbert_pretrained", bert_model="bert-base-uncased", task_name="subj", max_seq_length=64, do_lower_case=False, train_batch_size=32, learning_rate=5e-5, num_train_epochs=10.0, warmup_proportion=0.1, seed=42, save_every_epoch = True):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
        """
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.save_model_dir = save_model_dir
        self.bert_model = bert_model
        self.task_name = task_name
        self.max_seq_length = max_seq_length
        self.do_lower_case = do_lower_case
        self.train_batch_size = train_batch_size
        self.learning_rate = learning_rate
        self.num_train_epochs = num_train_epochs
        self.warmup_proportion = warmup_proportion
        self.seed = seed
        self.save_every_epoch = save_every_epoch
        self.sample_num = 1
        self.sample_ratio = 7
        self.gpu = 0
        self.temp = 1.0

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import shutil
import logging
import argparse
import random
import json
from tqdm import tqdm, trange

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW#, WarmupLinearSchedule
#import train_text_classifier_new

#import cbert_utils

"""initialize logger"""
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

"""cuda or cpu"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def main():
    #parser = argparse.ArgumentParser()

    ## Required parameters
    #parser.add_argument("--data_dir", default="datasets", type=str,
    #                    help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    #parser.add_argument("--output_dir", default="aug_data", type=str,
    #                    help="The output dir for augmented dataset.")
    #parser.add_argument("--save_model_dir", default="cbert_model", type=str,
    #                    help="The cache dir for saved model.")
    #parser.add_argument("--bert_model", default="bert-base-uncased", type=str,
    #                    help="The path of pretrained bert model.")
    #parser.add_argument("--task_name", default="subj", type=str,
    #                    help="The name of the task to train.")
    #parser.add_argument("--max_seq_length", default=64, type=int,
    #                    help="The maximum total input sequence length after WordPiece tokenization. \n"
    #                         "Sequence longer than this will be truncated, and sequences shorter \n"
    #                         "than this wille be padded.")
    #parser.add_argument("--do_lower_case", default=False, action='store_true',
    #                    help="Set this flag if you are using an uncased model.")
    #parser.add_argument("--train_batch_size", default=32, type=int,
    #                    help="Total batch size for training.")
    #parser.add_argument("--learning_rate", default=5e-5, type=float,
    #                    help="The initial learning rate for Adam.")
    #parser.add_argument("--num_train_epochs", default=10.0, type=float,
    #                    help="Total number of training epochs to perform.")
    #parser.add_argument("--warmup_proportion", default=0.1, type=float,
    #                    help="Proportion of training to perform linear learning rate warmup for."
    #                         "E.g., 0.1 = 10%% of training.")
    #parser.add_argument("--seed", type=int, default=42,
    #                    help="random seed for initialization")
    #parser.add_argument("--save_every_epoch", default=True, action='store_true')

    #args = parser.parse_args()

    with open("global.config", 'r') as f:
        configs_dict = json.load(f)
    
    print(configs_dict)
    args = Arguments()
    args.task_name = configs_dict.get("dataset")
    args.save_model_dir = "cbert_pretrained"
    print(args)
    
    """prepare processors"""
    augProcessor = AugProcessor()
    processors = {
        ## you can add your processor here
        "TREC": augProcessor,
        "stsa.fine": augProcessor,
        "stsa.binary": augProcessor,
        "mpqa": augProcessor,
        "rt-polarity": augProcessor,
        "subj": augProcessor,
        "amazon": augProcessor,
    }

    task_name = args.task_name
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
    processor = processors[task_name]
    print("task_name", task_name)
    label_list = processor.get_labels(task_name)
    print("label_list", label_list)

    """prepare model"""
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

    ## leveraging lastest bert module in Transformers to load pre-trained model tokenizer
    tokenizer = BertTokenizer.from_pretrained(args.bert_model)

    print("bert", args.bert_model)
    
    ## leveraging lastest bert module in Transformers to load pre-trained model (weights)
    model = BertForMaskedLM.from_pretrained(args.bert_model, output_hidden_states = True)

    if task_name == 'stsa.fine':
        model.bert.embeddings.token_type_embeddings = torch.nn.Embedding(5, 768)
        model.bert.embeddings.token_type_embeddings.weight.data.normal_(mean=0.0, std=0.02)
    elif task_name == 'TREC':
        model.bert.embeddings.token_type_embeddings = torch.nn.Embedding(6, 768)
        model.bert.embeddings.token_type_embeddings.weight.data.normal_(mean=0.0, std=0.02)
    elif task_name == 'amazon':
        model.bert.embeddings.token_type_embeddings = torch.nn.Embedding(13, 768)
        model.bert.embeddings.token_type_embeddings.weight.data.normal_(mean=0.0, std=0.02)

    args.data_dir = os.path.join(args.data_dir, task_name)
    args.output_dir = os.path.join(args.output_dir, task_name)
    os.makedirs(args.output_dir, exist_ok=True)

    print(args.data_dir)
    train_examples = processor.get_train_examples(args.data_dir)
    train_features, num_train_steps, train_dataloader = construct_train_dataloader(train_examples, label_list, args.max_seq_length, args.train_batch_size, args.num_train_epochs, tokenizer, device)
    
    model.cuda()
    #global_step = 0
    #train_features = convert_examples_to_features(
    #    train_examples, label_list, args.max_seq_length, tokenizer)
    #num_train_steps = int(len(train_features) / args.train_batch_size * args.num_train_epochs)
    
    #logger.info("***** Running training *****")
    #logger.info("  Num examples = %d", len(train_features))
    #logger.info("  Batch size = %d", args.train_batch_size)
    #logger.info("  Num steps = %d", num_train_steps)
    #all_init_ids = torch.tensor([f.init_ids for f in train_features], dtype=torch.long)
    #all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
    #all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
    #all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
    #all_masked_lm_labels = torch.tensor([f.masked_lm_labels for f in train_features], dtype=torch.long)
    #train_data = TensorDataset(all_init_ids, all_input_ids, all_input_mask, all_segment_ids, all_masked_lm_labels)
    #train_sampler = RandomSampler(train_data)
    #train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

    ## if you have a GPU, put everything on cuda
    #model.cuda()

    print("batch", args.train_batch_size)

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_features))
    logger.info("  Batch size = %d", args.train_batch_size)
    logger.info("  Num steps = %d", num_train_steps)

    ## in Transformers, optimizer and schedules are splitted and instantiated like this:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grounded_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grounded_parameters, lr=args.learning_rate, correct_bias=False)
    model.train()

    os.makedirs(args.save_model_dir, exist_ok=True)
    save_model_dir = os.path.join(args.save_model_dir, task_name)
    if not os.path.exists(save_model_dir):
        os.mkdir(save_model_dir)

    for e in trange(int(args.num_train_epochs), desc="Epoch"):
        avg_loss = 0.

        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.cuda() for t in batch) #.cuda()
            _, input_ids, input_mask, segment_ids, masked_ids, segment_span, class_span = batch
            """train generator at each batch"""
            #print(segment_span, class_span)
            optimizer.zero_grad() 
            outputs = model(input_ids, input_mask, segment_ids, labels=masked_ids)
            loss = outputs.loss#[0]
            loss.backward()
            avg_loss += loss.item()
            optimizer.step()
            if (step + 1) % 50 == 0:
                print("avg_loss: {}".format(avg_loss / 50))
                avg_loss = 0
        if args.save_every_epoch:
            save_model_name = "BertForMaskedLM_" + task_name + "_epoch_" + str(e + 1)
            save_model_path = os.path.join(save_model_dir, save_model_name)
            torch.save(model, save_model_path)
        else:
            if (e + 1) % 10 == 0:
                save_model_name = "BertForMaskedLM_" + task_name + "_epoch_" + str(e + 1)
                save_model_path = os.path.join(save_model_dir, save_model_name)
                torch.save(model, save_model_path)

if __name__ == "__main__":
    main()

{'dataset': 'subj', 'bilm_mode': 'sampling', 'stop_epoch': 20, 'bilm_wordwise': True, 'bilm_gumbel': False, 'no_label': False, 'dropout': 0.5, 'epoch': 100, 'bilm_dropout': 0.0, 'unit': 300, 'bilm_layer': 1, 'save_model': False, 'learning_rate': 0.0001, 'bilm_residual': 0.0, 'bilm_add_original': 0.0, 'bilm_temp': 1.0, 'out': 'result', 'validation': True, 'seed': 2018, 'bilm_ratio': 0.25, 'layer': 1, 'model': 'rnn', 'bilm': None, 'gpu': 0, 'bilm_unit': 1024, 'resume_vocab': None, 'batchsize': 64}
task_name subj
label_list ['0', '1']


KeyboardInterrupt: ignored

In [ ]:
path = '/content/cbert_pretrained/stsa.fine/'
file_dir = os.listdir(path)

with zipfile.ZipFile('pretrained_cbert.zip', mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    for dirpath, dirnames, filenames in os.walk(path):
      for filename in filenames:
        print("Файл:", os.path.join(dirpath, filename))
        add_file = os.path.join(dirpath, filename)
        zf.write(add_file)

os.system('pretrained_cbert.zip')

In [ ]:
shutil.move('/content/pretrained_cbert.zip', '/content/drive/MyDrive')

'/content/drive/MyDrive/pretrained_cbert.zip'

In [ ]:
zip_file = '/content/drive/MyDrive/pretrained_cbert.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

FileNotFoundError: ignored

In [ ]:
#test
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os
import shutil
import logging
import argparse
import random
from tqdm import tqdm, trange
import json

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW

#import cbert_utils
import train_text_classifier

#PYTORCH_PRETRAINED_BERT_CACHE = ".pytorch_pretrained_bert"

"""initialize logger"""
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

"""cuda or cpu"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def convert_ids_to_str(ids, tokenizer):
    """converts token_ids into str."""
    tokens = []
    for token_id in ids:
        token = tokenizer._convert_id_to_token(token_id)
        tokens.append(token)
    outputs = rev_wordpiece(tokens)
    return outputs

def main():
    #parser = argparse.ArgumentParser()

    ## required parameters
    #parser.add_argument("--data_dir", default="datasets", type=str,
    #                    help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    #parser.add_argument("--output_dir", default="aug_data", type=str,
    #                    help="The output dir for augmented dataset")
    #parser.add_argument("--save_model_dir", default="cbert_model", type=str,
    #                    help="The cache dir for saved model.")
    #parser.add_argument("--bert_model", default="bert-base-uncased", type=str,
    #                    help="The path of pretrained bert model.")
    #parser.add_argument("--task_name", default="subj",type=str,
    #                    help="The name of the task to train.")
    #parser.add_argument("--max_seq_length", default=64, type=int,
    #                    help="The maximum total input sequence length after WordPiece tokenization. \n"
    #                         "Sequences longer than this will be truncated, and sequences shorter \n"
    #                         "than this will be padded.")
    #parser.add_argument("--do_lower_case", default=False, action='store_true',
    #                    help="Set this flag if you are using an uncased model.")
    #parser.add_argument("--train_batch_size", default=32, type=int,
    #                    help="Total batch size for training.")
    #parser.add_argument("--learning_rate", default=5e-5, type=float,
    #                    help="The initial learning rate for Adam.")
    #parser.add_argument("--num_train_epochs", default=9.0, type=float,
    #                    help="Total number of training epochs to perform.")
    #arser.add_argument("--warmup_proportion", default=0.1, type=float,
    #                    help="Proportion of training to perform linear learning rate warmup for. "
    #                         "E.g., 0.1 = 10%% of training.")
    #parser.add_argument('--seed', default=42, type=int, 
    #                    help="random seed for initialization")
    #parser.add_argument('--sample_num', default=1, type=int,
    #                    help="sample number")
    #parser.add_argument('--sample_ratio', default=7, type=int,
    #                    help="sample ratio")
    #parser.add_argument('--gpu', default=0, type=int,
    #                    help="gpu id")
    #parser.add_argument('--temp', default=1.0, type=float,
    #                    help="temperature")
    

    #args = parser.parse_args()

    args = Arguments()
    args.save_model_dir = "/content/drive/MyDrive/pretrained_cbert"
    
    """prepare processors"""
    #augProcessor = AugProcessor()

    with open("global.config", 'r') as f:
        configs_dict = json.load(f)

    args = Arguments()
    args.task_name = configs_dict.get("dataset")
    args.output_dir = args.output_dir + '_{}_{}_{}_{}'.format(args.sample_num, args.sample_ratio, args.gpu, args.temp)
    print(args)
    
    """prepare processors"""
    augProcessor = AugProcessor() #cbert_utils.
    processors = {
        ## you can add your processor here
        "TREC": augProcessor,
        "stsa.fine": augProcessor,
        "stsa.binary": augProcessor,
        "mpqa": augProcessor,
        "rt-polarity": augProcessor,
        "subj": augProcessor,
        "amazon": augProcessor,
    }

    task_name = args.task_name
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))
    processor = processors[task_name]
    label_list = processor.get_labels(task_name)

    ## prepare for model
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

    def load_model(model_name):
        weights_path = os.path.join(args.save_model_dir, model_name)
        print("WP", weights_path)
        model = torch.load(weights_path)
        return model
    
    args.data_dir = os.path.join(args.data_dir, task_name)
    print("Input dir ", args.data_dir)
    args.output_dir = os.path.join(args.output_dir, task_name)
    print("Output dir ", args.output_dir)
    if os.path.exists(args.output_dir):
        shutil.rmtree(args.output_dir)
    shutil.copytree("aug_data/{}".format(task_name), args.output_dir)

    ## prepare for training
    train_examples = processor.get_train_examples(args.data_dir)
    train_features, num_train_steps, train_dataloader = \
        construct_train_dataloader(train_examples, label_list, args.max_seq_length, 
        args.train_batch_size, args.num_train_epochs, tokenizer, device)

    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Batch size = %d", args.train_batch_size)
    logger.info("  Num steps = %d", num_train_steps)

    save_model_dir = os.path.join(args.save_model_dir, task_name)
    if not os.path.exists(save_model_dir):
        os.mkdir(save_model_dir)
    MASK_id = convert_tokens_to_ids(['[MASK]'], tokenizer)[0]

    #origin_train_path = os.path.join(args.output_dir, "train_origin.tsv")
    save_train_path = os.path.join(args.output_dir, "train.tsv")
    #shutil.copy(origin_train_path, save_train_path)
    #best_test_acc = train_text_classifier.train("aug_data_{}_{}_{}_{}".format(args.sample_num, args.sample_ratio, args.gpu, args.temp))
    #print("before augment best acc:{}".format(best_test_acc))

    output_examples = []

    for e in trange(int(args.num_train_epochs), desc="Epoch"):
        torch.cuda.empty_cache()
        cbert_name = "{}/BertForMaskedLM_{}_epoch_{}".format(task_name.lower(), task_name.lower(), e+1)
        model = load_model(cbert_name)
        model.cuda()
        #shutil.copy(origin_train_path, save_train_path)
        #save_train_path = ''
        save_train_file = open(save_train_path, 'a')
        tsv_writer = csv.writer(save_train_file, delimiter='\t')
        for _, batch in enumerate(train_dataloader):
            model.eval()
            batch = tuple(t.cuda() for t in batch)
            init_ids, _, input_mask, segment_ids, _, segmnet_span, class_span = batch
            #input_lens = [sum(mask).item() for mask in input_mask]
            #masked_idx = np.squeeze([np.random.randint(0, l, max(l//args.sample_ratio, 1)) for l in input_lens])
            #for ids, idx in zip(init_ids, masked_idx):
            #    ids[idx] = MASK_id
            with torch.no_grad():
              predictions = model(init_ids, input_mask, segment_ids)
              hidden_states = predictions.hidden_states[2]
              #print("hidden_states", hidden_states.shape)
              sentence_embedding = torch.mean(hidden_states[-1], dim=0).squeeze()
              #print(sentence_embedding)
              #print(sentence_embedding.size())
              output_examples.append(OutputObject(embedding = sentence_embedding.detach().cpu().numpy(), segment = segmnet_span.detach().cpu().numpy(), cl = class_span.detach().cpu().numpy()))
              #print(output_examples)
              #np.savetxt('tensor.txt', sentence_embedding.detach().cpu().numpy())
              
          #    # get last four layers
          #    last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
          #    # cast layers to a tuple and concatenate over the last dimension
          #    cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
          #    print(cat_hidden_states.size())

              # take the mean of the concatenated vector over the token dimension
          #    cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
          #    print(cat_sentence_embedding)
          #    print(cat_sentence_embedding.size())  
            
            for out_ex in output_examples:
                #preds = torch.multinomial(preds, args.sample_num, replacement=True)[idx]
                #if len(preds.size()) == 2:
                #    preds = torch.transpose(preds, 0, 1)
                #for pred in preds:
                #    ids[idx] = pred
                #    new_str = convert_ids_to_str(ids.cpu().numpy(), tokenizer)
                tsv_writer.writerow(np.hstack([out_ex.embedding, int(out_ex.segment[0]), int(out_ex.cl[0])]))
            torch.cuda.empty_cache()
        
        #predictions = predictions.detach().cpu()
        #model.cpu()
        #torch.cuda.empty_cache()
        #bak_train_path = os.path.join(args.output_dir, "train_epoch_{}.tsv".format(e))
        #shutil.copy(save_train_path, bak_train_path)
        #best_test_acc = train_text_classifier.train("aug_data_{}_{}_{}_{}".format(args.sample_num, args.sample_ratio, args.gpu, args.temp))
        #print("epoch {} augment best acc:{}".format(e, best_test_acc))

if __name__ == "__main__":
    main()


Amazon


04/20/2021 11:25:55 - INFO - __main__ - [cbert] *** Example ***
04/20/2021 11:25:55 - INFO - __main__ - [cbert] guid: train-1
04/20/2021 11:25:55 - INFO - __main__ - [cbert] tokens: [CLS] school uniforms is good . [UNK] drag a student over the fence from outside the school grounds . [SEP]
04/20/2021 11:25:55 - INFO - __main__ - [cbert] init_ids: 101 2082 11408 2003 2204 1012 100 8011 1037 3076 2058 1996 8638 2013 2648 1996 2082 5286 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/20/2021 11:25:55 - INFO - __main__ - [cbert] input_ids: 101 2082 11408 2003 2204 1012 100 8011 103 3076 2058 1996 8638 2013 2648 1996 2082 5286 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/20/2021 11:25:55 - INFO - __main__ - [cbert] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/20/2021 11:25:55 - INFO - __mai

Input dir  datasets/amazon
Output dir  aug_data_1_7_0_1.0/amazon


Epoch:  10%|█         | 1/10 [00:00<00:07,  1.25it/s]

WP cbert_pretrained/amazon/BertForMaskedLM_amazon_epoch_1
WP cbert_pretrained/amazon/BertForMaskedLM_amazon_epoch_2


...

Epoch:  80%|████████  | 8/10 [00:05<00:01,  1.51it/s]

WP cbert_pretrained/amazon/BertForMaskedLM_amazon_epoch_9


Epoch:  90%|█████████ | 9/10 [00:06<00:00,  1.50it/s]

WP cbert_pretrained/amazon/BertForMaskedLM_amazon_epoch_10


Epoch: 100%|██████████| 10/10 [00:06<00:00,  1.50it/s]


In [ ]:
output_examples

NameError: ignored

In [ ]:
import numpy as np

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
loss